<a href="https://colab.research.google.com/github/Costa200513/Prediccion-de-supervivencia-en-el-Titanic-con-K-Nearest-Neighbors/blob/main/Challenge_Prediccion_de_Supervivencia_en_el_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Cargar el dataset**

In [ ]:
import pandas as pd
df = pd.read_csv('train.csv')

#**Ver cantidad de columnas y filas**

In [ ]:
df.shape

(891, 12)

#**Cambiar los datos tipo String por datos numericos**

In [ ]:
df['Sex'] = df ['Sex'].map ({'male': 0, 'female': 1})

df['Embarked'] = df ['Embarked'].map ({'S': 0, 'C': 1, 'Q': 2})

#**Ver los datos del dataset**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    float64
dtypes: float64(3), int64(6), object(3)
memory usage: 83.7+ KB


#**Ver los datos nulos**

In [ ]:
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


#**Completar los datos de edad con la media**

In [ ]:
df['Age'].fillna(df['Age'].median(), inplace=True)
df.isnull().sum()

/tmp/ipython-input-4086271142.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)


,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,0
SibSp,0
Parch,0
Ticket,0
Fare,0


#**Eliminar la columna Cabin**

In [ ]:
df.drop(columns='Cabin', inplace=True)
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,0
SibSp,0
Parch,0
Ticket,0
Fare,0


#**Definir variables a utilizar**

In [ ]:
caracteristicas  = ['Age', 'Sex','Pclass']
X = df[caracteristicas]

y= df['Survived']

#**Definir datos de entrenamiento y prueba**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)


#**Scaler**

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)


#**Funcionamiento del modelo**

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

modelo = KNeighborsClassifier(n_neighbors=5)
modelo.fit(X_train, y_train)
prediccion = modelo.predict(X_val)
accuracy = accuracy_score(y_val, prediccion)
print('Exactitud en el conjunto de validación:', accuracy)


Exactitud en el conjunto de validación: 0.7653631284916201


#**Cargar el dataset de testeo**

In [ ]:
test_data = pd.read_csv('test.csv')
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


#**Modificaciones necesarias al datset de testeo para funcionar correctamente**

In [ ]:
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)
test_data['Sex'] = test_data ['Sex'].map ({'male': 0, 'female': 1})


/tmp/ipython-input-1497445345.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_data['Age'].fillna(test_data['Age'].median(), inplace=True)


#**Generacion de archivo para prueba final**

In [ ]:
X_val = test_data[caracteristicas]
X_val = scaler.transform(X_val)
prediccion = modelo.predict(X_val)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': prediccion})
output.to_csv('submission.csv', index=False)